In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from dateutil import parser
from datetime import datetime
import json

In [2]:
df1 = pd.read_csv("../dataset/Records_TRAIN.csv", sep="|")

In [3]:
df1.head()

,ACCT_NBR,SUCCESSOR,VOLTAGE,PARNO,XRHSH,CONTRACT_CAPACITY,ACCT_CONTROL,ACCT_WGS84_X,ACCT_WGS84_Y
0,000A31FE38C6F648845941B642E287FE0200000000004A...,1,LOW,0,2,12.0,0,"53,422551543686701","19,592757873078753"
1,000A31FE38C6F648845941B642E287FE0200000000008D...,1,LOW,0,2,12.0,0,"53,407108521588633","19,569743639932696"
2,000A31FE38C6F648845941B642E287FE0200000000008D...,3,LOW,2,2,25.0,0,"53,407108521588633","19,569743639932696"
3,000A31FE38C6F648845941B642E287FE0200000000008D...,4,LOW,2,2,25.0,0,"53,407108521588633","19,569743639932696"
4,000A31FE38C6F648845941B642E287FE0200000000008D...,5,LOW,2,2,25.0,0,"53,407108521588633","19,569743639932696"


In [4]:
len(df1)

1234509

In [5]:
df1.dtypes

ACCT_NBR              object
SUCCESSOR              int64
VOLTAGE               object
PARNO                  int64
XRHSH                  int64
CONTRACT_CAPACITY    float64
ACCT_CONTROL           int64
ACCT_WGS84_X          object
ACCT_WGS84_Y          object
dtype: object

In [6]:
df1.ACCT_NBR.nunique()

490664

In [7]:
acct_nbr_to_id = {acct_nbr: i for i, acct_nbr in enumerate(df1['ACCT_NBR'].unique())}

In [8]:
len(acct_nbr_to_id.keys())

490664

In [9]:
df1['ACCT_NBR'] = df1['ACCT_NBR'].map(acct_nbr_to_id)

In [10]:
df1.dtypes

ACCT_NBR               int64
SUCCESSOR              int64
VOLTAGE               object
PARNO                  int64
XRHSH                  int64
CONTRACT_CAPACITY    float64
ACCT_CONTROL           int64
ACCT_WGS84_X          object
ACCT_WGS84_Y          object
dtype: object

In [11]:
df1.to_csv("TRAIN_Records.csv", index=False)

In [12]:
df2 = pd.read_csv("../dataset/Consumptions_TRAIN.csv", sep="|")

In [13]:
df2.head()

,ACCT_NBR,SUCCESSOR,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,000A31FE38C6F648845941B642E287FE02000000B7602D...,1,13,000A31FE38C6F648845941B642E287FE0200000032011D...,56,2019-08-26
1,000A31FE38C6F648845941B642E287FE020000003CC307...,1,11,000A31FE38C6F648845941B642E287FE020000002BFDE8...,2236,2022-03-10
2,000A31FE38C6F648845941B642E287FE02000000543AF1...,1,13,000A31FE38C6F648845941B642E287FE0200000055887E...,678,2021-04-20
3,000A31FE38C6F648845941B642E287FE02000000617012...,1,13,000A31FE38C6F648845941B642E287FE02000000310114...,156,2022-04-27
4,000A31FE38C6F648845941B642E287FE020000006CBF0C...,1,13,000A31FE38C6F648845941B642E287FE02000000B94B46...,502,2022-06-03


In [14]:
len(df2)

9201394

In [15]:
df2.ACCT_NBR.nunique()

459826

In [16]:
df2.MS_METER_NBR.nunique()

475643

In [17]:
df2_filtered = df2[~df2['ACCT_NBR'].isin(acct_nbr_to_id.keys())]

In [18]:
len(df2_filtered)

682

In [19]:
df2_filtered.ACCT_NBR.nunique()

389

In [20]:
unique_acct_nbrs = df2['ACCT_NBR'].unique().tolist()

In [21]:
i=0
for acct_nbr in unique_acct_nbrs:
    if acct_nbr not in acct_nbr_to_id:
        i+=1
        new_id = max(acct_nbr_to_id.values()) + 1
        acct_nbr_to_id[acct_nbr] = new_id
print(i)

389


In [22]:
len(acct_nbr_to_id)

491053

In [23]:
df2['ACCT_NBR'] = df2['ACCT_NBR'].map(acct_nbr_to_id)

In [24]:
df2.head()

,ACCT_NBR,SUCCESSOR,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,329855,1,13,000A31FE38C6F648845941B642E287FE0200000032011D...,56,2019-08-26
1,113886,1,11,000A31FE38C6F648845941B642E287FE020000002BFDE8...,2236,2022-03-10
2,219271,1,13,000A31FE38C6F648845941B642E287FE0200000055887E...,678,2021-04-20
3,245307,1,13,000A31FE38C6F648845941B642E287FE02000000310114...,156,2022-04-27
4,250806,1,13,000A31FE38C6F648845941B642E287FE02000000B94B46...,502,2022-06-03


In [25]:
mse_meter_nbr_to_id = {
    mse_meter_nbr: i for i, mse_meter_nbr in enumerate(df2['MS_METER_NBR'].unique())
}

In [26]:
len(mse_meter_nbr_to_id)

475643

In [27]:
df2['MS_METER_NBR'] = df2['MS_METER_NBR'].map(mse_meter_nbr_to_id)

In [28]:
df2['MEASUREMENT_DATE'] = pd.to_datetime(df2['MEASUREMENT_DATE'])

In [29]:
min(df2['MEASUREMENT_DATE']), max(df2['MEASUREMENT_DATE'])

(Timestamp('2018-01-01 00:00:00'), Timestamp('2022-10-01 00:00:00'))

In [30]:
df2.head()

,ACCT_NBR,SUCCESSOR,BS_RATE,MS_METER_NBR,CSS_MS_HS_USE,MEASUREMENT_DATE
0,329855,1,13,0,56,2019-08-26
1,113886,1,11,1,2236,2022-03-10
2,219271,1,13,2,678,2021-04-20
3,245307,1,13,3,156,2022-04-27
4,250806,1,13,4,502,2022-06-03


In [31]:
df2.dtypes

ACCT_NBR                     int64
SUCCESSOR                    int64
BS_RATE                     object
MS_METER_NBR                 int64
CSS_MS_HS_USE                int64
MEASUREMENT_DATE    datetime64[ns]
dtype: object

In [32]:
df2.to_csv("TRAIN_Consumptions.csv", index=False)

In [33]:
df3 = pd.read_csv("../dataset/PowerThefts_TRAIN.csv", sep="|")

In [34]:
df3.head(2)

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
0,000A31FE38C6F648845941B642E287FE02000000004D63...,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2018-03-01,01/03/2018,01/03/2018,1283.0
1,000A31FE38C6F648845941B642E287FE0200000000801E...,8,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2018-08-16,01112018,01112018,640.0


In [35]:
len(df3)

3841

In [36]:
df3.ACCT_NBR.nunique()

3600

In [37]:
df3.dtypes

ACCT_NBR                   object
SUCCESSOR                   int64
ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ          object
ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ          object
HMANAF                     object
INITIAL_DETECTION_DATE     object
DETECTION_DATE             object
NON_REGISTERED_ENERGY     float64
dtype: object

In [38]:
df3_filtered = df3[~df3['ACCT_NBR'].isin(acct_nbr_to_id.keys())]
len(df3_filtered)

109

In [39]:
df3_filtered.ACCT_NBR.nunique()

106

In [40]:
unique_acct_nbrs = df3['ACCT_NBR'].unique().tolist()
i=0
for acct_nbr in unique_acct_nbrs:
    if acct_nbr not in acct_nbr_to_id:
        i+=1
        new_id = max(acct_nbr_to_id.values()) + 1
        acct_nbr_to_id[acct_nbr] = new_id
print(i)

106


In [41]:
len(acct_nbr_to_id)

491159

In [42]:
df3['ACCT_NBR'] = df3['ACCT_NBR'].map(acct_nbr_to_id)

In [43]:
df3.head()

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
0,416,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2018-03-01,01/03/2018,01/03/2018,1283.0
1,678,8,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2018-08-16,01112018,01112018,640.0
2,957,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2018-12-05,18/12/2018,18/12/2018,4665.0
3,20201,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2017-01-19,30/01/2017,30/01/2017,NaN
4,2140,5,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-05-29,29/05/2019,29/05/2019,1049.0


In [44]:
def parse_date(date_str):
    try:
        return parser.parse(date_str)
    except:
        formats = ['%d%m%Y', '%Y-%m-%d', '%Y/%m/%d', '%d-%b-%Y', '%d-%b-%y']
        for fmt in formats:
            try:
                return datetime.strptime(date_str, fmt)
            except:
                pass
        return pd.NaT

In [45]:
tmp_df3 = df3.copy()

In [46]:
tmp_df3['DETECTION_DATE'] = tmp_df3['DETECTION_DATE'].apply(parse_date)

In [47]:
tmp_df3.head()

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
0,416,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2018-03-01,01/03/2018,2018-01-03,1283.0
1,678,8,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2018-08-16,01112018,2018-11-01,640.0
2,957,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2018-12-05,18/12/2018,2018-12-18,4665.0
3,20201,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2017-01-19,30/01/2017,2017-01-30,NaN
4,2140,5,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-05-29,29/05/2019,2019-05-29,1049.0


In [48]:
tmp_df3[tmp_df3.DETECTION_DATE.isnull()]

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,05/022019,NaT,0.0
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,12/102017,NaT,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,11/042022,NaT,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,15/042022,NaT,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,18/032019,NaT,1447.0
3663,491155,3,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-02-14,NaN,NaT,NaN


In [49]:
len(tmp_df3[tmp_df3.DETECTION_DATE.isnull()])

6

In [50]:
indices = [15, 902, 1360, 1570, 2930, 3663]

In [51]:
selected_rows = df3.iloc[indices]

In [52]:
selected_rows

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,05/022019,05/022019,0.0
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,12/102017,12/102017,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,11/042022,11/042022,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,15/042022,15/042022,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,18/032019,18/032019,1447.0
3663,491155,3,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-02-14,NaN,NaN,NaN


In [53]:
indices = [15, 902, 1360, 1570, 2930]
selected_rows = df3.iloc[indices]
selected_rows

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,05/022019,05/022019,0.0
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,12/102017,12/102017,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,11/042022,11/042022,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,15/042022,15/042022,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,18/032019,18/032019,1447.0


In [54]:
selected_rows['DETECTION_DATE'] = selected_rows['DETECTION_DATE'].apply(lambda x: datetime.strptime(x, '%d/%m%Y').strftime('%Y-%m-%d'))

/tmp/ipykernel_4210/2314740787.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['DETECTION_DATE'] = selected_rows['DETECTION_DATE'].apply(lambda x: datetime.strptime(x, '%d/%m%Y').strftime('%Y-%m-%d'))


In [55]:
selected_rows

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,05/022019,2019-02-05,0.0
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,12/102017,2017-10-12,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,11/042022,2022-04-11,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,15/042022,2022-04-15,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,18/032019,2019-03-18,1447.0


In [56]:
tmp_df3.loc[indices] = selected_rows

In [57]:
tmp_df3[tmp_df3.DETECTION_DATE.isnull()]

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
3663,491155,3,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-02-14,NaN,NaT,NaN


In [58]:
df3 = tmp_df3.copy()

In [59]:
tmp_df3['INITIAL_DETECTION_DATE'] = tmp_df3['INITIAL_DETECTION_DATE'].apply(parse_date)

In [60]:
tmp_df3[tmp_df3.INITIAL_DETECTION_DATE.isnull()]

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,NaT,2019-02-05,0.0
176,40750,1,Μέσω Καταμέτρησης (ΔΑΚ),ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2016-04-28,NaT,2016-07-27,NaN
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,NaT,2017-10-12,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,NaT,2022-04-11,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,NaT,2022-04-15,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,NaT,2019-03-18,1447.0
3663,491155,3,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-02-14,NaT,NaT,NaN


In [61]:
len(tmp_df3[tmp_df3.INITIAL_DETECTION_DATE.isnull()])

7

In [62]:
indices = [15, 176, 902, 1360, 1570, 2930, 3663]

In [63]:
selected_rows = df3.iloc[indices]
selected_rows

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,05/022019,2019-02-05,0.0
176,40750,1,Μέσω Καταμέτρησης (ΔΑΚ),ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2016-04-28,28/042016,2016-07-27,NaN
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,12/102017,2017-10-12,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,11/042022,2022-04-11,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,15/042022,2022-04-15,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,18/032019,2019-03-18,1447.0
3663,491155,3,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-02-14,NaN,NaT,NaN


In [64]:
indices = [15, 176, 902, 1360, 1570, 2930]

In [65]:
selected_rows = df3.iloc[indices]
selected_rows

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,05/022019,2019-02-05,0.0
176,40750,1,Μέσω Καταμέτρησης (ΔΑΚ),ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2016-04-28,28/042016,2016-07-27,NaN
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,12/102017,2017-10-12,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,11/042022,2022-04-11,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,15/042022,2022-04-15,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,18/032019,2019-03-18,1447.0


In [66]:
selected_rows['INITIAL_DETECTION_DATE'] = selected_rows['INITIAL_DETECTION_DATE'].apply(lambda x: datetime.strptime(x, '%d/%m%Y').strftime('%Y-%m-%d'))

/tmp/ipykernel_4210/2133048324.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_rows['INITIAL_DETECTION_DATE'] = selected_rows['INITIAL_DETECTION_DATE'].apply(lambda x: datetime.strptime(x, '%d/%m%Y').strftime('%Y-%m-%d'))


In [67]:
selected_rows

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
15,491054,6,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-02-05,2019-02-05,2019-02-05,0.0
176,40750,1,Μέσω Καταμέτρησης (ΔΑΚ),ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2016-04-28,2016-04-28,2016-07-27,NaN
902,216405,3,Μέσω Στοχευμένων Ελέγχων,ΑΚΡΟΔΕΚΤΕΣ,2017-10-12,2017-10-12,2017-10-12,3121.0
1360,327574,4,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-04-07,2022-04-11,2022-04-11,NaN
1570,470407,2,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-03-22,2022-04-15,2022-04-15,NaN
2930,237605,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-03-18,2019-03-18,2019-03-18,1447.0


In [68]:
tmp_df3.loc[indices] = selected_rows

In [69]:
tmp_df3[tmp_df3.DETECTION_DATE.isnull()]

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
3663,491155,3,ΑΓΝΩΣΤΟΣ,ΑΚΡΟΔΕΚΤΕΣ,2022-02-14,NaT,NaT,NaN


In [70]:
tmp_df3['HMANAF'] = tmp_df3['HMANAF'].apply(parse_date)

In [71]:
tmp_df3[tmp_df3.HMANAF.isnull()]

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY


In [72]:
tmp_df3.head()

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
0,416,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2018-03-01,2018-01-03,2018-01-03,1283.0
1,678,8,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2018-08-16,2018-11-01,2018-11-01,640.0
2,957,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2018-12-05,2018-12-18,2018-12-18,4665.0
3,20201,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2017-01-19,2017-01-30,2017-01-30,NaN
4,2140,5,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-05-29,2019-05-29,2019-05-29,1049.0


In [73]:
len(tmp_df3)

3841

In [74]:
tmp_df3.head(16)

,ACCT_NBR,SUCCESSOR,ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ,ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ,HMANAF,INITIAL_DETECTION_DATE,DETECTION_DATE,NON_REGISTERED_ENERGY
0,416,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2018-03-01,2018-01-03,2018-01-03,1283.0
1,678,8,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2018-08-16,2018-11-01,2018-11-01,640.0
2,957,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2018-12-05,2018-12-18,2018-12-18,4665.0
3,20201,2,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2017-01-19,2017-01-30,2017-01-30,NaN
4,2140,5,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-05-29,2019-05-29,2019-05-29,1049.0
5,2140,5,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΓΝΩΣΤΟ,2019-05-29,2019-05-29,2019-05-29,236.0
6,2333,3,Μέσω Λοιπών Εργασιών & Συντήρησης,ΜΗΧΑΝΙΣΜΟΣ ΜΕΤΡΗΣΗΣ,2018-05-16,2021-05-19,2021-05-19,9242.0
7,2437,4,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2019-09-10,2019-11-18,2019-11-18,3015.0
8,2497,1,Μέσω Λοιπών Εργασιών & Συντήρησης,ΑΚΡΟΔΕΚΤΕΣ,2018-08-14,2018-10-22,2018-10-22,NaN
9,2645,3,Μέσω Καταμέτρησης (ΔΑΚ),ΑΚΡΟΔΕΚΤΕΣ,2019-03-18,2019-06-12,2019-06-12,NaN


In [75]:
tmp_df3.dtypes

ACCT_NBR                           int64
SUCCESSOR                          int64
ΤΡΟΠΟΣ_ΕΝΤΟΠΙΣΜΟΥ                 object
ΣΗΜΕΙΟ_ΕΝΤΟΠΙΣΜΟΥ                 object
HMANAF                    datetime64[ns]
INITIAL_DETECTION_DATE    datetime64[ns]
DETECTION_DATE            datetime64[ns]
NON_REGISTERED_ENERGY            float64
dtype: object

In [76]:
tmp_df3.to_csv("TRAIN_PowerThefts.csv", index=False)

In [77]:
df4 = pd.read_csv("../dataset/Representations_TRAIN.csv", sep="|")

In [78]:
df4.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
0,000A31FE38C6F648845941B642E287FE0200000043989C...,4,W,NaN,2017-04-04,2022-07-19
1,000A31FE38C6F648845941B642E287FE020000004398BA...,1,Δ,NaN,1995-10-18,1999-12-22
2,000A31FE38C6F648845941B642E287FE020000004398BA...,4,Δ,Δ,2019-11-06,2020-06-23
3,000A31FE38C6F648845941B642E287FE020000004398BA...,5,Δ,NaN,2020-06-23,2022-09-16
4,000A31FE38C6F648845941B642E287FE020000004398BD...,1,Δ,Δ,1995-10-15,2000-01-13


In [79]:
df4.ACCT_NBR.nunique()

492339

In [80]:
df4.dtypes

ACCT_NBR       object
SUCCESSOR       int64
SUPPLIER       object
SUPPLIER_TO    object
START_DATE     object
END_DATE       object
dtype: object

In [81]:
df4_filtered = df4[~df4['ACCT_NBR'].isin(acct_nbr_to_id.keys())]
len(df4_filtered)

1915

In [82]:
df4_filtered.ACCT_NBR.nunique()

1914

In [83]:
unique_acct_nbrs = df4['ACCT_NBR'].unique().tolist()

In [84]:
i=0
for acct_nbr in unique_acct_nbrs:
    if acct_nbr not in acct_nbr_to_id:
        i+=1
        new_id = max(acct_nbr_to_id.values()) + 1
        acct_nbr_to_id[acct_nbr] = new_id
print(i)

1914


In [85]:
len(acct_nbr_to_id)

493073

In [86]:
df4['ACCT_NBR'] = df4['ACCT_NBR'].map(acct_nbr_to_id)

In [87]:
df4.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
0,123622,4,W,NaN,2017-04-04,2022-07-19
1,123623,1,Δ,NaN,1995-10-18,1999-12-22
2,123623,4,Δ,Δ,2019-11-06,2020-06-23
3,123623,5,Δ,NaN,2020-06-23,2022-09-16
4,123624,1,Δ,Δ,1995-10-15,2000-01-13


In [88]:
df4['START_DATE'] = df4['START_DATE'].apply(parse_date)

In [89]:
df4.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
0,123622,4,W,NaN,2017-04-04 00:00:00,2022-07-19
1,123623,1,Δ,NaN,1995-10-18 00:00:00,1999-12-22
2,123623,4,Δ,Δ,2019-11-06 00:00:00,2020-06-23
3,123623,5,Δ,NaN,2020-06-23 00:00:00,2022-09-16
4,123624,1,Δ,Δ,1995-10-15 00:00:00,2000-01-13


In [90]:
min(df4.START_DATE)

datetime.datetime(1199, 11, 22, 0, 0)

In [91]:
tmp_df4 = df4.copy()

In [92]:
import datetime as dt

In [93]:
tmp_df4['START_DATE'] = tmp_df4['START_DATE'].replace(
    dt.datetime(1199, 11, 22), dt.datetime(1999, 11, 22)
)                                    

In [94]:
min(tmp_df4.START_DATE)

datetime.datetime(1199, 12, 8, 0, 0)

In [95]:
tmp_df4['START_DATE'] = tmp_df4['START_DATE'].replace(
    dt.datetime(1199, 12, 8), dt.datetime(1999, 12, 8)
)  

In [96]:
min(tmp_df4.START_DATE)

Timestamp('1983-12-31 00:00:00')

In [97]:
tmp_df4[tmp_df4.START_DATE.isnull()]

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE


In [98]:
tmp_df4['END_DATE'] = tmp_df4['END_DATE'].apply(parse_date)

In [99]:
min(tmp_df4.END_DATE)

Timestamp('1995-08-09 00:00:00')

In [100]:
tmp_df4[tmp_df4.END_DATE.isnull()]

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE


In [101]:
tmp_df4.dtypes

ACCT_NBR                int64
SUCCESSOR               int64
SUPPLIER               object
SUPPLIER_TO            object
START_DATE     datetime64[ns]
END_DATE       datetime64[ns]
dtype: object

In [102]:
df4=tmp_df4.copy()

In [103]:
df4.head()

,ACCT_NBR,SUCCESSOR,SUPPLIER,SUPPLIER_TO,START_DATE,END_DATE
0,123622,4,W,NaN,2017-04-04,2022-07-19
1,123623,1,Δ,NaN,1995-10-18,1999-12-22
2,123623,4,Δ,Δ,2019-11-06,2020-06-23
3,123623,5,Δ,NaN,2020-06-23,2022-09-16
4,123624,1,Δ,Δ,1995-10-15,2000-01-13


In [104]:
df4.to_csv("TRAIN_Representations.csv", index=False)

In [105]:
df5 = pd.read_csv("../dataset/Requests_TRAIN.csv", sep="|")

In [106]:
df5.ACCT_NBR.nunique()

653066

In [107]:
df5_filtered = df5[~df5['ACCT_NBR'].isin(acct_nbr_to_id.keys())]
len(df5_filtered)

1529521

In [108]:
df5_filtered.ACCT_NBR.nunique()

426677

In [109]:
unique_acct_nbrs = df5_filtered['ACCT_NBR'].unique().tolist()

In [115]:
len(acct_nbr_to_id)

873101

In [116]:
i=0
for j, acct_nbr in enumerate(unique_acct_nbrs):
    if j % 100_000 == 0:
        print("J", j)
    if acct_nbr not in acct_nbr_to_id:
        i+=1
        new_id = max(acct_nbr_to_id.values()) + 1
        acct_nbr_to_id[acct_nbr] = new_id
        if len(acct_nbr_to_id) % 10000 == 0:
            print(len(acct_nbr_to_id))
print(i)

J 0
J 100000
J 200000
J 300000
880000
890000
J 400000
900000
910000
46649


In [117]:
len(acct_nbr_to_id)

919750

In [118]:
df5['ACCT_NBR'] = df5['ACCT_NBR'].map(acct_nbr_to_id)

In [119]:
df5.to_csv("Train_Requests.csv", index=False)

In [120]:
with open('acct_nbr_to_id.txt', 'w') as f:
     f.write(json.dumps(acct_nbr_to_id))
with open('ms_meter_nbr_to_id.txt', 'w') as f:
     f.write(json.dumps(mse_meter_nbr_to_id))